In [6]:
%pip install crewai
%pip install duckduckgo-search

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Local Open Source Models

crewAI supports integration with local models, thorugh tools such as Ollama, for enhanced flexibility and customization. This allows you to utilize your own models, which can be particularly useful for specialized tasks or data privacy concerns.

## Setting Up Ollama
* 
Install Ollama: Ensure that Ollama is properly installed in your environment. Follow the installation guide provided by Ollama for detailed instructions
* 
Configure Ollama: Set up Ollama to work with your local model. You will probably need to tweak the model using a Modelfile. I'd recommend adding Observation as a stop word and playing with top_p ant temperature.

# Integrating Ollama with CrewAI
* 
Instantiate Ollama Model: Create an instance of the Ollama model. You can specify the model and the base URL during instantiation. For example:

In [3]:
from langchain.llms import Ollama
# Pass Ollama Model to Agents: When creating your agents within the CrewAI framework, you can pass the Ollama model as an argument to the Agent constructor. For instance:

In [4]:
import os
from crewai import Agent, Task, Crew, Process

In [5]:
# You can choose to use a local model through Ollama for example.
# In this case we will use OpenHermes 2.5 as an example.
#
from langchain.llms import Ollama
ollama_llm = Ollama(model="openhermes")

# If you are using an ollama like above you don't need to set OPENAI_API_KEY.
#os.environ["OPENAI_API_KEY"] = "Your Key"

# Define your tools, custom or not.
# Install duckduckgo-search for this example:
#
# !pip install -U duckduckgo-search

from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [ ]:
# Define your agents with roles and goals
researcher = Agent(
  role='Senior Research Analyst',
  goal='Uncover cutting-edge developments in AI and data science in',
  backstory="""You are a Senior Research Analyst at a leading tech think tank.
  Your expertise lies in identifying emerging trends and technologies in AI and
  data science. You have a knack for dissecting complex data and presenting
  actionable insights.""",
  verbose=True,
  allow_delegation=False,
  tools=[search_tool],
  llm=ollama_llm

)
writer = Agent(
  role='Tech Content Strategist',
  goal='Craft compelling content on tech advancements',
  backstory="""You are a renowned Tech Content Strategist, known for your insightful
  and engaging articles on technology and innovation. With a deep understanding of
  the tech industry, you transform complex concepts into compelling narratives.""",
  verbose=True,
  llm=ollama_llm,
  allow_delegation=True
)

# Create tasks for your agents
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Compile your findings in a detailed report. Your final answer MUST be a full analysis report""",
  agent=researcher
)

task2 = Task(
  description="""Using the insights from the researcher's report, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Aim for a narrative that captures the essence of these breakthroughs and their
  implications for the future. Your final answer MUST be the full blog post of at least 3 paragraphs.""",
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
  process=Process.sequential # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

Working Agent: Senior Research Analyst
Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Compile your findings in a detailed report. Your final answer MUST be a full analysis report ...


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: duckduckgo_search
Action Input: "Latest advancements in AI 2024"published 27 December 2023 While 2023 was the year of AI, 2024 will be the year we use it. Comments (0) (Image credit: Aston Martin/Apple) We have just come to the end of a year where... In 2024, multimodal AI is expected to evolve significantly, ushering in a shift in generative AI capabilities. These models are progressing beyond traditional single-mode functions,... This year's trends reflect a deepening sophistication and caution in AI development and deployment strategies, with an eye to ethics, safety and the evolving regula

In [ ]:
# Get your crew to work!
result = crew.kickoff()

print("######################")
print(result)